# GNN Training

In [1]:

import torch
from torch_geometric.loader import DataLoader

from GNN_model.HeteroGAT import HeteroGAT
from GNN_model.trainer import Trainer
from GNN_model.graph_processor import GraphProcessor

from sklearn.model_selection import train_test_split


## Data Preparation

In [2]:
load_saved_data = False
base_data_dir = 'data/example_pool_sc_1/'
data = GraphProcessor(base_data_dir, load_saved_data=load_saved_data).load_and_process_graphs()
loader = DataLoader(data, batch_size=1, shuffle=False)

In [3]:
def get_tensor(size, flag):
    if flag:
        return torch.ones(size, dtype=torch.bool)
    else:
        return torch.zeros(size, dtype=torch.bool)

def get_masks(data):
    indices = list(range(len(data)))
    train_idx, test_idx = train_test_split(indices, test_size=.2)
    train_idx, val_idx = train_test_split(train_idx, test_size=.1)
    return train_idx, val_idx, test_idx

masks = get_masks(data)

## Model

In [4]:
num_classes = 2
hidden_channels = 64
epochs = 200

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model = HeteroGAT(hidden_channels, num_classes).to(device=device)

In [7]:
criterion = torch.nn.BCELoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Define optimizer.

In [8]:
trainer = Trainer(f'{base_data_dir}/models/', loader, device, masks)
trainer.train(model, criterion, optimizer)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not dict

In [ ]:
# TODO hyperparameter tuning
# TODO visualize results